In [41]:
import pandas as pd
#akshit_df = './mlchallenge_set_2021.tsv'
#akshit_valid = './mlchallenge_set_validation.tsv'
#sam_df = './drive/MyDrive/mlchallenge_set_2021_edited.txt'
#sam_valid = './mlchallenge_set_validation.tsv'
SA_valid=pd.read_table('/Users/shivankagrawal/Documents/ebay/mlchallenge_set_validation.tsv',header=None)
SA_df=pd.read_table('/Users/shivankagrawal/Documents/ebay/mlchallenge_set_2021.tsv',header=None)
df=SA_df
valid=SA_valid
#df = pd.read_table(sam_df)
#valid = pd.read_table(sam_valid,sep='\t')
df.columns=['category','primary_image_url','All Links','Tags','index']
#valid.columns=['ID', 'Group']

# Parsing Logic #

In [42]:
import re
from collections import Counter
freq=Counter()
attribute=[['']]*len(df)
trialrange=3000
for x in range(trialrange):#range(int(len(df)/10)):#len(df)
    attribute[x]=df.iloc[x,3].lower()
    attribute[x] = re.sub(r'[()]','', attribute[x])
    attribute[x] = re.split(r',', attribute[x])
    attribute[x] = [a.split(':') for a in attribute[x]]
    freq+=Counter([i[0] for i in attribute[x]])
    tempdict={}
    for i in attribute[x]:

            try:
                tempdict[i[0]]=float(i[1])
            except:
                try:
                    tempdict[i[0]]=i[1]
                except:
                    pass
    attribute[x]=tempdict

df['seg']=attribute
#print(df['seg'])

In [43]:
for i in df['seg'][0:trialrange]:
  if 'color' in i:
    i['color']=re.split(r'[-\s,/]', i['color'])
    print(i['color'])

te']
['blue']
['red']
['multi', 'color']
['green']
['red']
['dark', 'grey', 'mint', 'candy', 'cool', 'grey', 'green']
['lbrown', 'cwhite', 'lbrown']
['blue']
['black']
['black']
['black']
['white']
['black']
['silver']
['green']
['blue']
['black', 'grey', 'red', 'white']
['gray']
['red']
['yellow']
['wine']
['blue']
['vivid', 'pink', 'gum', 'light', 'brown']
['multi', 'color']
['green']
['black', 'white']
['multi', 'color']
['multi', 'color']
['white']
['white']
['red']
['black']
['black']
['black']
['gray']
['white']
['blue']
['multi', 'color']
['black']
['multi', 'color']
['grays']
['whites']
['black', 'fire', 'red', 'metalic', 'silver', 'white']
['red', 'white', 'and', 'blue']
['purple']
['college', 'navy']
['gray']
['blue']
['teal']
['black']
['blue']
['wolf', 'grey', 'black', 'white']
['brown']
['yellow']
['black']
['black', 'gray', 'red']
['tropical', 'teal', 'sonic', 'yellow', 'blueprint']
['black']
['grey']
['multi', 'color']
['grey', 'black']
['red', 'white', 'blue']
['blue']


In [44]:
Brands=[]
Images=[]
Colors = []
color_images = []
simple_colors = {'black','white','red','blue','green','yellow','brown','purple','pink','gray','grey'}
print(trialrange)
for i in range(trialrange):
    try:
        for word in df['seg'].iloc[i]['color']:
            if word in simple_colors:
              Colors.append(word)
              color_images.append(df['primary_image_url'].iloc[i])
              break
        #df['seg'].iloc[i]['brand']
        #df['primary_image_url'].iloc[i]
        #print(df['seg'].iloc[i]['brand'])
        #print(df['primary_image_url'].iloc[i])
        #if df['seg'].iloc[i]['brand'] == 'nike' or df['seg'].iloc[i]['brand'] == 'adidas':
        #   Brands.append(df['seg'].iloc[i]['brand'])
        #    Images.append(df['primary_image_url'].iloc[i])
        '''
        for key in simple_colors:
          if key in df['seg'].iloc[i]['color']:
              Colors.append(key)
              color_images.append(df['primary_image_url'].iloc[i])
              continue
         '''   
    except:
        continue
        #Brands.remove[-1]
        #print('not possible at: ',i)
print(Colors)
#print(color_images)

3000
['gray', 'white', 'green', 'blue', 'green', 'black', 'black', 'black', 'black', 'black', 'black', 'blue', 'black', 'blue', 'black', 'white', 'blue', 'red', 'green', 'black', 'white', 'gray', 'white', 'red', 'white', 'brown', 'white', 'red', 'black', 'red', 'blue', 'black', 'black', 'black', 'gray', 'purple', 'blue', 'white', 'red', 'white', 'purple', 'purple', 'white', 'black', 'red', 'red', 'white', 'black', 'grey', 'black', 'brown', 'blue', 'black', 'yellow', 'white', 'red', 'white', 'white', 'white', 'black', 'blue', 'black', 'blue', 'black', 'black', 'black', 'black', 'white', 'black', 'green', 'black', 'black', 'black', 'blue', 'black', 'purple', 'gray', 'gray', 'white', 'red', 'white', 'pink', 'yellow', 'white', 'blue', 'blue', 'black', 'gray', 'white', 'white', 'white', 'blue', 'red', 'black', 'black', 'brown', 'blue', 'red', 'black', 'blue', 'black', 'red', 'red', 'white', 'gray', 'black', 'white', 'gray', 'black', 'black', 'white', 'red', 'black', 'red', 'blue', 'black', 

In [53]:
n = 500
import time
from PIL import Image, ImageFile
import requests
from io import BytesIO
import numpy as np
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_array = []
images = []
max_height = 0
max_width = 0
i = 0
for img in color_images[0:n]:
    response = requests.get(img)
    if i%200 == 0:
        print(i)
    i+=1
    img = Image.open(BytesIO(response.content))
    img=img.resize((200,200))
    image_array.append(img.convert('RGB'))
    #img.show()
    #time.sleep(5)
    pix=np.asarray(img)
    pix=pix.astype('float32')
    pix/=255.0
    if pix.shape[1] > max_width:
        max_width = pix.shape[1]
    if pix.shape[0] > max_height:
        max_height = pix.shape[0]
    pix
    images.append(pix)
i=0
'''
for img in images:
    if i%200 == 0:
        print(i)
    i+=1
    
    
    try:
        img = img.convert('RGB')
        margin = Image.new(img.mode, (max_width, max_height), (64,64,64))
    except:
        img = img.convert('RGB')
        margin = Image.new(img.mode,(max_width,max_height),(64,64,64))
    margin.paste(img, (0, 0))
    image_array.append(np.asarray(margin))
    '''

0


ConnectionError: HTTPSConnectionPool(host='i.ebayimg.com', port=443): Max retries exceeded with url: /00/s/MTYwMFgxMjAw/z/RroAAOSwMD5a-iUq/$_12.JPG?set_id=880000500F (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x141c27df0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
#ignore this
n = 500
from PIL import Image, ImageFile
import requests
import urllib.request
from io import BytesIO
import numpy as np
import grequests
ImageFile.LOAD_TRUNCATED_IMAGES = True
image_array = []
images = []
max_height = 0
max_width = 0
i = 0
urls = []
for img in color_images[0:n]:
    #response = requests.get(img)
    urls.append(urllib.request.urlretrieve(img))
    #urls.append(img)
print(urls)
glist = (grequests.get(u) for u in urls)
rs = grequests.map(glist)
print(rs)
for g in rs:
    if i%200 == 0:
        print(i)
    i+=1
    img = Image.open(BytesIO(g.content))
    if np.asarray(img).shape[1] > max_width:
        max_width = np.asarray(img).shape[1]
    if np.asarray(img).shape[0] > max_height:
        max_height = np.asarray(img).shape[0]
    images.append(img)



i=0
#code to make images all same size: right now uses max_width and max_height
for img in images:
    if i%200 == 0:
        print(i)
    i+=1
    try:
        img = img.convert('RGB')
        margin = Image.new(img.mode, (max_width, max_height), (64,64,64))
    except:
        img = img.convert('RGB')
        margin = Image.new(img.mode,(max_width,max_height),(64,64,64))
    margin.paste(img, (0, 0))
    image_array.append(np.asarray(margin))

In [ ]:
n = 500
from PIL import Image, ImageFile
import requests as requests
from io import BytesIO
import numpy as np


ImageFile.LOAD_TRUNCATED_IMAGES = True
image_array = []
images = []
max_height = 0
max_width = 0
i = 0

for img in color_images[0:n]:
  f.append(session.get(img))
for i in as_completed(f):
  print(i)
  image_array.append(i.result())
print(image_array)

# Neural Net Training #

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dropout

model = models.Sequential()
model.add(layers.Conv2D(4, (3, 3), activation='relu', input_shape=(max_width, max_height, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='relu'))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
brand_dict = {}
num = 0
labels = []
for b in Colors:
    if b not in brand_dict:
        brand_dict[b] = num
        num+=1
    labels.append(brand_dict[b])
print(len(labels))
m = 500
n = round(m*.8)
#train_images = np.asarray(image_array[0:n])
#test_images = np.asarray(image_array[n:m])
#train_labels = np.asarray(labels[0:n])
#test_labels = np.asarray(labels[n:m])
#print(type(test_images[0]))

In [ ]:
history = model.fit(np.asarray(image_array[0:500]), np.asarray(labels[0:500]), epochs=30, batch_size = 10,validation_split=.8)